In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

KeyboardInterrupt: ignored

In [1]:
import pandas as pd
data_load = pd.read_csv("/content/sample_data/data4.csv")

data_load.head()

,tokens,vec,genre_ind
0,"[7548, 4844, 5497, 1954, 1007, 10096, 8314, 54...","[1, 0, 0, 0, 0, 0]",0
1,"[9757, 12907, 12640, 745, 15875, 7807, 8307, 1...","[0, 0, 0, 0, 0, 1]",5
2,"[14123, 7602, 12117, 19279, 15143, 12034, 1877...","[0, 0, 0, 0, 0, 1]",5
3,"[2536, 16182, 16671, 5034, 12907, 13487, 7785,...","[1, 0, 0, 0, 0, 0]",0
4,"[16942, 9449, 15636, 12473, 667, 6428, 2869, 8...","[0, 1, 0, 0, 0, 0]",1


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv("/content/sample_data/movies_metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# "장르" column 정리
genres = []
for list_ in data["genres"].apply(lambda x: eval(x)):
    genres.extend(list_) if len(list_) > 0 else genres.append(np.nan)
    
# Unique Genre-Id Tuple
genres_unique = [set(tuple(x.items()) if type(x) == dict else None for x in genres)]

In [0]:
# retain genre by name

genre_byname = data["genres"].apply(lambda x: [*map(lambda y: y["name"], eval(x))] if len(eval(x)) > 0 else np.nan)

In [0]:
# generate new_data with necessary data

new_data = pd.DataFrame(columns=["original_title","overview","genres"])

for i in new_data.columns:
  new_data[i] = data[i].copy()

new_data["genres"] = genre_byname

new_data = new_data.dropna().reset_index(drop=True)

In [0]:
# extract genres by single list (set(all_genres)=unique set of genres)
all_genres = []
for i in new_data["genres"]:
  all_genres.extend(i)

In [7]:
# exclude insignificant genres
exclude_list = []
genres_dict = dict()
for i in set(all_genres):
    print("Genre: ",i, "\nCount: ",all_genres.count(i))
    genres_dict[i] = all_genres.count(i)
    print("-"*15)
    if all_genres.count(i) == 1:
        exclude_list.append(i)

exclude_list.extend(["TV Movie","Foreign", "Animation"])

Genre:  Telescene Film Group Productions 
Count:  1
---------------
Genre:  Western 
Count:  1035
---------------
Genre:  War 
Count:  1310
---------------
Genre:  Sentai Filmworks 
Count:  1
---------------
Genre:  Drama 
Count:  20023
---------------
Genre:  Music 
Count:  1588
---------------
Genre:  Aniplex 
Count:  1
---------------
Genre:  The Cartel 
Count:  1
---------------
Genre:  Pulser Productions 
Count:  1
---------------
Genre:  Fantasy 
Count:  2290
---------------
Genre:  Vision View Entertainment 
Count:  1
---------------
Genre:  Animation 
Count:  1920
---------------
Genre:  Romance 
Count:  6673
---------------
Genre:  History 
Count:  1379
---------------
Genre:  TV Movie 
Count:  751
---------------
Genre:  GoHands 
Count:  1
---------------
Genre:  Science Fiction 
Count:  3028
---------------
Genre:  Crime 
Count:  4269
---------------
Genre:  Adventure 
Count:  3470
---------------
Genre:  Odyssey Media 
Count:  1
---------------
Genre:  Foreign 
Count:  1599

In [0]:
genres_sorted = [j[0] for j in sorted(genres_dict.items(), key = lambda x: x[1])]
exclude_list = genres_sorted[:-5]
#exclude_list.extend(["Documentary","Horror"])

In [0]:
exclude_list.remove("Documentary")
exclude_list.remove("Horror")

In [0]:
genres_excluded = []
for i in new_data["genres"]:
    new = list(set(i) - set(exclude_list))
    genres_excluded.append(new if len(new) > 0 else np.nan)

In [11]:
genres_excluded

[['Comedy'],
 nan,
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Thriller', 'Drama', 'Action'],
 ['Comedy', 'Romance'],
 ['Drama', 'Action'],
 ['Thriller', 'Action'],
 ['Thriller', 'Action'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 nan,
 ['Drama'],
 ['Action'],
 ['Drama'],
 ['Drama', 'Romance'],
 ['Comedy'],
 ['Comedy'],
 ['Comedy', 'Action'],
 ['Comedy', 'Thriller'],
 ['Thriller', 'Drama'],
 ['Thriller', 'Action'],
 ['Thriller', 'Drama'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Comedy', 'Drama'],
 ['Drama', 'Romance'],
 nan,
 ['Drama'],
 ['Drama'],
 ['Thriller'],
 ['Comedy', 'Drama'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Drama'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance'],
 ['Drama'],
 ['Drama'],
 ['Drama', 'Action'],
 ['Drama', 'Romance'],
 ['Action'],
 ['Comedy', 'Drama', 'Thriller'],
 ['Drama', 'Romance'],
 ['Thriller'],
 ['Drama'],
 ['Drama', 'Romance'],
 ['Thriller', 'Drama'],
 ['Thriller', 'Drama', 'Action'],
 ['Comedy', 'R

In [0]:
new_data["genres"] = genres_excluded
new_data = new_data.dropna().reset_index(drop=True)

In [0]:
# 소문자 전환
#new_data["overview"] = new_data["overview"].apply(lambda x: x.lower())

In [0]:
# delete url links & drop "No overview"

import re

row_drop = []
for e,i in enumerate(new_data["overview"]):
    new_data["overview"][e] = re.sub('(https?://)[^ ]+','', i)
    new_data["overview"][e] = re.sub('[^ ]+(www.)+[^ ]+','', i)
    new_data["overview"][e] = re.sub('hancinema.net[^ ]+','', i)
    new_data["overview"][e] = re.sub('(mc)[^ ]+','',i)
    if "No overview" in i or "No Overview" in i or "no overview" in i:
        row_drop.append(e)
        
new_data = new_data.drop(row_drop).reset_index(drop=True)

In [15]:
# tokenize

import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize

ov_token = new_data["overview"].apply(word_tokenize)


# Use RegX

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
ov_token_reg = new_data["overview"].apply(tokenizer.tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = list(set(stopwords.words('english')))

stopWords.extend(["s","movie","movies","film","films","meet",
                  "mr","ms","mrs"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# exclude stopwords

ov_token_sw = []
for word_list in ov_token_reg:
    ov_token_sw.append([word for word in word_list if word not in stopWords])

In [0]:
# Stemming 

import nltk
stemmer = nltk.stem.porter.PorterStemmer()

train_X_raw = []

for i in ov_token_sw:
    train_X_raw.append([stemmer.stem(words) for words in i])

In [19]:
train_X_raw[0]

['led',
 'woodi',
 'andi',
 'toy',
 'live',
 'happili',
 'room',
 'andi',
 'birthday',
 'bring',
 'buzz',
 'lightyear',
 'onto',
 'scene',
 'afraid',
 'lose',
 'place',
 'andi',
 'heart',
 'woodi',
 'plot',
 'buzz',
 'but',
 'circumst',
 'separ',
 'buzz',
 'woodi',
 'owner',
 'duo',
 'eventu',
 'learn',
 'put',
 'asid',
 'differ']

In [20]:
data["overview"]

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45463    When one of her hits goes wrong, a professiona...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 45466, dtype: object

In [21]:
whole=[]
for num,i in enumerate(train_X_raw):
    whole+=i
    if num%10000==0:
        print(num)
from collections import Counter
counter=Counter(whole)
L=dict(counter)
def f(x):
    return x[1]
D = sorted(L.items(),key=f,reverse=True)
stops = dict(D[:5000]).keys()

0
10000
20000
30000


In [0]:
# exclusion
train_X_raw = pd.Series(train_X_raw)
train_X_clean = train_X_raw.apply(lambda i: [word for word in i if word in stops])

word_exclude_list = ["one","two","three","four","five",
                     "six","seven","eight","nine","ten",
                     "much","would","ever","whose","also",
                     "seem","jame","john","david","peter","michael", 
                     "mari", "ann", "lee", "u", "ii", "sam", 
                    "els", "lisa", 'charl', 'linda', 'daniel', 
                     'steven', 'steve', 'joe', 'kevin', 'william', 
                     'harri', 'richard', 'elizabeth', 'martin', 
                     'jim', 'anna']

#word_exclude_list += list(stops)


train_X_final = train_X_clean.apply(lambda i: [word for word in i if word not in word_exclude_list])

In [0]:

flat_tokens = []
for i in train_X_final:
    flat_tokens.extend(i)


In [24]:
len(set(flat_tokens))

4959

In [0]:
# create word 2 index dictionary for mapping later

def word_indexing(flat_tokens):
    wd2ix = dict()
    ix2wd = dict()

    cnt = 1
    for words in set(flat_tokens):
        wd2ix[words] = cnt
        ix2wd[cnt] = words
        cnt+=1
    return wd2ix, ix2wd


def token_cutting(tokens, wd2ix, cut_tokens=60, padding=0):
    tokens_aft_cut = []
    for i in tokens:
        if len(i) >= cut_tokens:
            tokens_aft_cut.append([wd2ix[word] if word in wd2ix.keys() else 0 for word in i[:cut_tokens]])
        elif len(i) < cut_tokens:
            new_i = [wd2ix[word] for word in i]
            new_i += [padding for k in range(cut_tokens-len(i))]
            tokens_aft_cut.append(new_i)
    return pd.Series(tokens_aft_cut)

In [0]:
tokens = token_cutting(train_X_final, word_indexing(flat_tokens)[0], cut_tokens = 50)

In [0]:
data_load["tokens"] = tokens

In [28]:
data_load

,tokens,vec,genre_ind
0,"[1603, 2608, 404, 3494, 784, 369, 89, 404, 389...","[1, 0, 0, 0, 0, 0]",0
1,"[1563, 1286, 1468, 364, 4908, 1588, 3664, 646,...","[0, 0, 0, 0, 0, 1]",5
2,"[2941, 1781, 4139, 3879, 2026, 4900, 776, 2967...","[0, 0, 0, 0, 0, 1]",5
3,"[3976, 3910, 1139, 169, 827, 1468, 1599, 339, ...","[1, 0, 0, 0, 0, 0]",0
4,"[110, 1490, 80, 179, 512, 1921, 3855, 248, 141...","[0, 1, 0, 0, 0, 0]",1
...,...,...,...
39629,"[1563, 780, 4260, 4251, 3252, 2180, 1281, 3101...","[0, 0, 0, 1, 0, 0]",3
39630,"[1322, 814, 1508, 1624, 241, 1479, 4774, 2354,...","[0, 1, 0, 0, 0, 0]",1
39631,"[2495, 3540, 2787, 2937, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1]",5
39632,"[3153, 579, 3651, 2657, 2169, 1745, 434, 1518,...","[0, 0, 0, 0, 0, 1]",5


In [0]:
from torch.utils.data import DataLoader, Dataset
class MovieData(Dataset):
    
    def __init__(self, data, max_len=60):
        super(MovieData, self).__init__()
        self.dataset = data
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, ix):
        text = self.dataset["tokens"][ix]
        #text_len = text.index(0) if 0 in text else 60
        label = self.dataset["genre_ind"][ix]
        text = torch.LongTensor(text)
        return text, label


In [0]:
# Original code is from https://github.com/spro/practical-pytorch
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

#from name_dataset import NameDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Parameters and DataLoaders
embedding_size = 500
batch_size = 300
HIDDEN_SIZE = 100
N_CHARS =  len(set(flat_tokens)) + 1 # ASCII
N_CLASSES = 6


class RNNClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        
        input = input.t()

        embedded = self.embedding(input)

        hidden = self._init_hidden(input.size(1))

        output, hidden = self.gru(embedded, hidden)

        fc_output = self.fc(output[-1,:,:]).reshape(input.size(1),N_CLASSES)
        #fc_output = hidden[-1]
        return torch.sigmoid(fc_output)

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return Variable(hidden)

In [0]:
#inputs = Variable(torch.LongTensor(train_X.tolist()))
#labels = Variable(torch.LongTensor(train_y.tolist()))

classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_CLASSES)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)


In [0]:
train_dataset = MovieData(data_load.iloc[:-int(len(data_load)/9),:].reset_index(drop=True))
test_dataset = MovieData(data_load.iloc[-int(len(data_load)/9):,:].reset_index(drop=True))
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = 1,
                                          shuffle = True)

In [35]:
loss_list = []
acc_list = []
print('Training...')
#labels = labels.squeeze(1)
for epoch in range(50):
    for i, (texts, labels) in enumerate(train_loader):
        target = Variable(torch.LongTensor(labels))
        
        optimizer.zero_grad()
        
        outputs = classifier(texts)
        
        loss = criterion(outputs,target)
        
        loss.backward()
        
        optimizer.step()
    
        
        train_y = pd.Series(labels.tolist())
        acc = (pd.Series(torch.argmax(outputs,1).tolist()) == train_y.reset_index(drop=True)).sum() / len(train_y)
        
        if i % 100 == 0:
            print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))
            print("Step {} Accuracy: {}".format(i, acc))
        
        acc_list.append(acc)
        loss_list.append(loss.item())
        
        
    loss.detach()


print("Learning finished!")

Training...
epoch: 1, loss: 1.196
Step 0 Accuracy: 0.7566666666666667


KeyboardInterrupt: ignored

In [0]:
GENRES = ["Comedy", "Action", "Thriller","Horror","Documentary", "Drama/Romance"]

In [0]:
test_acc_list = []
for i, (texts, labels) in enumerate(test_loader):
  target = Variable(torch.LongTensor(labels))
  
  outputs = classifier(texts)
  
  train_y = pd.Series(labels.tolist())
  acc = torch.argmax(outputs,1).tolist() == labels.tolist()
  
  if i % 100 == 0:
      print(GENRES[torch.argmax(outputs,1).tolist()[0]], GENRES[labels.tolist()[0]])
      print("Step {} Accuracy: {}".format(i, acc))
  
  test_acc_list.append(acc)
        

Drama/Romance Horror
Step 0 Accuracy: False
Drama/Romance Comedy
Step 100 Accuracy: False
Horror Horror
Step 200 Accuracy: True
Drama/Romance Drama/Romance
Step 300 Accuracy: True
Action Action
Step 400 Accuracy: True
Drama/Romance Comedy
Step 500 Accuracy: False
Horror Horror
Step 600 Accuracy: True
Horror Horror
Step 700 Accuracy: True
Action Action
Step 800 Accuracy: True
Drama/Romance Thriller
Step 900 Accuracy: False
Documentary Comedy
Step 1000 Accuracy: False
Documentary Documentary
Step 1100 Accuracy: True
Documentary Comedy
Step 1200 Accuracy: False
Drama/Romance Documentary
Step 1300 Accuracy: False
Horror Action
Step 1400 Accuracy: False
Comedy Drama/Romance
Step 1500 Accuracy: False
Action Drama/Romance
Step 1600 Accuracy: False
Drama/Romance Drama/Romance
Step 1700 Accuracy: True
Drama/Romance Comedy
Step 1800 Accuracy: False
Comedy Comedy
Step 1900 Accuracy: True
Comedy Drama/Romance
Step 2000 Accuracy: False
Drama/Romance Documentary
Step 2100 Accuracy: False
Drama/Roman

In [0]:
print("Test Accuracy: ", sum(test_acc_list) / len(test_acc_list))

Test Accuracy:  0.5448557801498978


In [0]:
train_dataset

In [0]:
def nlp_master(try_input):

  try_input = try_input.lower()

  # Use RegX

  from nltk.tokenize import RegexpTokenizer

  tokenizer = RegexpTokenizer(r'\w+')
  ov_token_reg = tokenizer.tokenize(try_input)

  # exclude stopwords

  ov_token_sw_try = [word for word in ov_token_reg if word not in stopWords]

  train_X_raw_try = [stemmer.stem(i) for i in ov_token_sw_try]

  train_X_clean_try = [word for word in train_X_raw_try if word in stops]

  word_exclude_list = ["one","two","three","four","five",
                      "six","seven","eight","nine","ten",
                      "much","would","ever","whose","also",
                      "seem","jame","john","david","peter","michael", 
                      "mari", "ann", "lee", "u", "ii", "sam", 
                      "els", "lisa", 'charl', 'linda', 'daniel', 
                      'steven', 'steve', 'joe', 'kevin', 'william', 
                      'harri', 'richard', 'elizabeth', 'martin', 
                      'jim', 'anna']

  #word_exclude_list += list(stops)

  train_X_final_try = [word for word in train_X_clean_try if word not in word_exclude_list]
  aa = token_cutting([train_X_final_try], word_indexing(flat_tokens)[0], cut_tokens = 50)
  output_try = classifier(Variable(torch.LongTensor(aa)))
  #print(GENRES[torch.argmax(output_try,1).tolist()[0]])
  return GENRES[torch.argmax(output_try,1).tolist()[0]]

In [39]:
nlp_master("I killed a hound by the street. The hound returned as a ghost and haunted me for the rest of my life")

'Horror'

In [40]:
nlp_master("Anne and Tom fell in love during high school. They went to university together, but during university, they broke up. Fortunately, in the end, they got married together and lived happily ever after")

'Drama/Romance'

In [41]:
nlp_master('April 6th, 1917. As a regiment assembles to wage war deep in enemy territory, two soldiers are assigned to race against time and deliver a message that will stop 1,600 men from walking straight into a deadly trap.')

'Action'

In [80]:
nlp_master("After school, Jim found a dead body in the park. Turns out the body was his ex-girlfriend. Shocked and frightened Jim tries to find who the killer would be. Something seems real suspicious about his current girlfirend's father")

'Horror'

In [81]:
nlp_master("After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to reverse Thanos' actions and restore balance to the universe.")

'Action'

In [82]:
nlp_master("After a strange foreigner (June Kunimura) appears, the village is overturned by a mysterious chain of events. The police make a tentative conclusion with collective wild mushroom poisoning, Rumors and suspicions spread all the way that the cause of every incident is due to the stranger. The police “Jong-gu” (Kwak Do-won) met the woman “Mown” (Chun Woo-hee) who witnessed the scene. Starts convincing rumors about outsiders. 'Jung-gu' became urgent when his daughter Hyo-jin (Kim Hwan-hee) began to get sick with symptoms similar to the victims. They hunt for foreigners and bring in shamanic sunshine, Hwang Jung-min.")

'Horror'

In [83]:
nlp_master("At the age of 21, Tim discovers he can travel in time and change what happens and has happened in his own life. His decision to make his world a better place by getting a girlfriend turns out not to be as easy as you might think.")

'Drama/Romance'